#Importing the libraries

In [69]:
# Importing necessary libraries

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#from imblearn.over_sampling import SMOTE  # imblearn library can be installed using pip install imblearn
#from imblearn.pipeline import Pipeline
#from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import chi2
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

#Examining the data

In [70]:
df = pd.read_csv('/content/drive/MyDrive/house-prices-advanced-regression-techniques/kg_train.csv')

In [71]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [72]:
# (rows , columns)
df.shape

(1460, 81)

In [73]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [74]:
# Shows the number of missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

#Feature Engineering

##Handling Missing values

In [75]:
# Getting the columns with missing values

index_misval = [3, 6, 25, 26, 30, 31, 32, 33,35,
                42, 57, 58, 59, 60, 63, 64, 72, 73, 74]

mis_val_cols = {}
colname = ''

for i in index_misval:
  colname = df.columns[i]
  mis_val_cols[i] = colname

mis_val_cols

{3: 'LotFrontage',
 6: 'Alley',
 25: 'MasVnrType',
 26: 'MasVnrArea',
 30: 'BsmtQual',
 31: 'BsmtCond',
 32: 'BsmtExposure',
 33: 'BsmtFinType1',
 35: 'BsmtFinType2',
 42: 'Electrical',
 57: 'FireplaceQu',
 58: 'GarageType',
 59: 'GarageYrBlt',
 60: 'GarageFinish',
 63: 'GarageQual',
 64: 'GarageCond',
 72: 'PoolQC',
 73: 'Fence',
 74: 'MiscFeature'}

In [76]:
# Getting the unique values for each of those columns

for k,v in mis_val_cols.items():
  print(v , df[v].unique())

LotFrontage [ 65.  80.  68.  60.  84.  85.  75.  nan  51.  50.  70.  91.  72.  66.
 101.  57.  44. 110.  98.  47. 108. 112.  74. 115.  61.  48.  33.  52.
 100.  24.  89.  63.  76.  81.  95.  69.  21.  32.  78. 121. 122.  40.
 105.  73.  77.  64.  94.  34.  90.  55.  88.  82.  71. 120. 107.  92.
 134.  62.  86. 141.  97.  54.  41.  79. 174.  99.  67.  83.  43. 103.
  93.  30. 129. 140.  35.  37. 118.  87. 116. 150. 111.  49.  96.  59.
  36.  56. 102.  58.  38. 109. 130.  53. 137.  45. 106. 104.  42.  39.
 144. 114. 128. 149. 313. 168. 182. 138. 160. 152. 124. 153.  46.]
Alley [nan 'Grvl' 'Pave']
MasVnrType ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
MasVnrArea [1.960e+02 0.000e+00 1.620e+02 3.500e+02 1.860e+02 2.400e+02 2.860e+02
 3.060e+02 2.120e+02 1.800e+02 3.800e+02 2.810e+02 6.400e+02 2.000e+02
 2.460e+02 1.320e+02 6.500e+02 1.010e+02 4.120e+02 2.720e+02 4.560e+02
 1.031e+03 1.780e+02 5.730e+02 3.440e+02 2.870e+02 1.670e+02 1.115e+03
 4.000e+01 1.040e+02 5.760e+02 4.430e+02 4.680e+02 6

In [77]:
# 'Lotfrontage' is the distance of the front of the property from the road
# So 'nan' would suggest that the property is not connected any street, hence
# 'nan' values should be filled with 0
df['LotFrontage'] = df['LotFrontage'].fillna(0)

In [78]:
# 'Alley' describes the kind of alley access to the property
df['Alley'] = df['Alley'].fillna('None')

# 'MasVnrType' refers to the if the external walls has any non-structural masonry finishing
df['MasVnrType'] = df['MasVnrType'].fillna('None')

# 'MasVnrArea' This represent the area the masonry finishing covers
# So if there is no finishing the answer here will be zero
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

# The 'BsmtQual' and 'BsmtCond' variables refer to the Basement height and condition respectively 
# And 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2' All describe the Basement

df['BsmtQual'] = df['BsmtQual'].fillna('None')
df['BsmtCond'] = df['BsmtCond'].fillna('None')
df['BsmtExposure'] = df['BsmtExposure'].fillna('None')
df['BsmtFinType1'] = df['BsmtFinType1'].fillna('None')
df['BsmtFinType2'] = df['BsmtFinType2'].fillna('None')

In [79]:
# The 'Eletrical' variable has only one missing value
# In this case it we can impute the mode for the 'nan' value
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])

In [80]:
df['FireplaceQu'] = df['FireplaceQu'].fillna('None')
df['GarageType'] = df['GarageType'].fillna('None')

In [81]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(df['GarageYrBlt'].median())
df['GarageFinish'] = df['GarageFinish'].fillna('None')
df['GarageQual'] = df['GarageQual'].fillna('None')
df['GarageCond'] = df['GarageCond'].fillna('None')
df['PoolQC'] = df['PoolQC'].fillna('None')
df['Fence'] = df['Fence'].fillna('None')
df['MiscFeature'] = df['MiscFeature'].fillna('None')

##Integer (Ranking Ordinal data) and One-Hot Encoding (Nominal data)

In [82]:
df.info() # All nan values has been handled

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [83]:
# In this section we have to transform textual data (norminal and ordinal)
# So that afterwards, we can plot a correlation heatmap to decide on the features that will go in to the model

# Grouping the data by their datatypes
dict_df_datatypes = df.columns.to_series().groupby(df.dtypes).groups
dict_df_datatypes

{int64: ['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'], float64: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], object: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'Garage

In [84]:
dict_df_datatypes = {k.name: v for k, v in dict_df_datatypes.items()}
dict_df_datatypes

{'float64': Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object'),
 'int64': Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
        'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
        'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
        'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
       dtype='object'),
 'object': Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'Bsm

The rule here is to use One-Hot encoding for nominal data

And Integer encoding (Ranking) for ordinal data starting from 0


In [85]:
ordinal_data = ['LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 
                'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2',
                'HeatingQC', 'Electrical', 'KitchenQual', 'Functional', 
                'FireplaceQu', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive',
                'PoolQC', 'Fence', 'BsmtExposure']

for i in ordinal_data:
  print(i,df[i].unique())

LotShape ['Reg' 'IR1' 'IR2' 'IR3']
Utilities ['AllPub' 'NoSeWa']
LandSlope ['Gtl' 'Mod' 'Sev']
ExterQual ['Gd' 'TA' 'Ex' 'Fa']
ExterCond ['TA' 'Gd' 'Fa' 'Po' 'Ex']
BsmtQual ['Gd' 'TA' 'Ex' 'None' 'Fa']
BsmtCond ['TA' 'Gd' 'None' 'Fa' 'Po']
BsmtFinType1 ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' 'None' 'LwQ']
BsmtFinType2 ['Unf' 'BLQ' 'None' 'ALQ' 'Rec' 'LwQ' 'GLQ']
HeatingQC ['Ex' 'Gd' 'TA' 'Fa' 'Po']
Electrical ['SBrkr' 'FuseF' 'FuseA' 'FuseP' 'Mix']
KitchenQual ['Gd' 'TA' 'Ex' 'Fa']
Functional ['Typ' 'Min1' 'Maj1' 'Min2' 'Mod' 'Maj2' 'Sev']
FireplaceQu ['None' 'TA' 'Gd' 'Fa' 'Ex' 'Po']
GarageFinish ['RFn' 'Unf' 'Fin' 'None']
GarageQual ['TA' 'Fa' 'Gd' 'None' 'Ex' 'Po']
GarageCond ['TA' 'Fa' 'None' 'Gd' 'Po' 'Ex']
PavedDrive ['Y' 'N' 'P']
PoolQC ['None' 'Ex' 'Fa' 'Gd']
Fence ['None' 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']
BsmtExposure ['No' 'Gd' 'Mn' 'Av' 'None']


In [86]:
df['LotShape'] = df['LotShape'].map({'Reg': 3,'IR1': 2, 'IR2': 1, 'IR3': 0})

df['Utilities'] = df['Utilities'].map({'AllPub': 1, 'NoSeWa': 0})

df['LandSlope'] = df['LandSlope'].map({'Gtl': 2, 'Mod': 1, 'Sev': 0})

df['ExterQual'] = df['ExterQual'].map({'Ex': 3, 'Gd': 2, 'TA': 1, 'Fa': 0})

df['ExterCond'] = df['ExterCond'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0})

df['BsmtQual'] = df['BsmtQual'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0, 'None': 0})

df['BsmtCond'] = df['BsmtCond'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0, 'None': 0})

df['BsmtFinType1'] = df['BsmtFinType1'].map({'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 2,
                                            'LwQ': 1, 'Unf': 0, 'None': 0})

df['BsmtFinType2'] = df['BsmtFinType2'].map({'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 2,
                                            'LwQ': 1, 'Unf': 0, 'None': 0})

df['HeatingQC'] = df['HeatingQC'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Po':1, 'Fa': 0})

df['Electrical'] = df['Electrical'].map({'SBrkr': 4, 'FuseA': 3,
                                         'FuseF': 2, 'FuseP': 1, 'Mix': 0})

df['KitchenQual'] = df['KitchenQual'].map({'Ex': 3, 'Gd': 2, 'TA': 1, 'Fa': 0})

df['Functional'] = df['Functional'].map({'Typ': 6, 'Min1': 5, 'Min2': 4,
                                         'Mod': 3, 'Maj1': 2, 'Maj2': 1, 'Sev': 0})

df['FireplaceQu'] = df['FireplaceQu'].map({'Ex': 4, 'Gd': 3, 'TA': 2,
                                           'Fa': 1, 'Po': 0, 'None': 0})

df['GarageFinish'] = df['GarageFinish'].map({'Fin': 2, 'RFn': 1, 'Unf': 0, 'None': 0})

df['GarageQual'] = df['GarageQual'].map({'Ex': 4, 'Gd': 3, 'TA': 2,
                                         'Fa': 1, 'Po': 0, 'None': 0})

df['GarageCond'] = df['GarageCond'].map({'Ex': 4, 'Gd': 3, 'TA': 2,
                                         'Fa': 1, 'Po': 0, 'None': 0})

df['PavedDrive'] = df['PavedDrive'].map({'Y': 2,'P': 1, 'N': 0})

df['PoolQC'] = df['PoolQC'].map({'Ex': 4, 'Gd': 3, 'TA': 2, 'Fa': 1, 'Po': 0, 'None': 0})

df['Fence'] = df['Fence'].map({'GdPrv': 4, 'MnPrv': 3, 'GdWo': 2, 'MnWw': 1, 'None': 0})

df['BsmtExposure'] = df['BsmtExposure'].map({'Gd': 4, 'Av': 3, 'Mn': 2, 'No': 1, 'None': 0})

In [87]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,None,3,Lvl,1,Inside,2,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,2,2,PConc,3,2,1,5,706,0,0,150,856,GasA,...,Y,4,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,0,Attchd,2003.0,1,2,548,2,2,2,0,61,0,0,0,0,0,0,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,3,Lvl,1,FR2,2,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,1,2,CBlock,3,2,4,4,978,0,0,284,1262,GasA,...,Y,4,1262,0,0,1262,0,1,2,0,3,1,1,6,6,1,2,Attchd,1976.0,1,2,460,2,2,2,298,0,0,0,0,0,0,0,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,2,Lvl,1,Inside,2,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,2,2,PConc,3,2,2,5,486,0,0,434,920,GasA,...,Y,4,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,2,Attchd,2001.0,1,2,608,2,2,2,0,42,0,0,0,0,0,0,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,2,Lvl,1,Corner,2,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,1,2,BrkTil,2,3,1,4,216,0,0,540,756,GasA,...,Y,4,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,3,Detchd,1998.0,0,3,642,2,2,2,0,35,272,0,0,0,0,0,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,2,Lvl,1,FR2,2,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,2,2,PConc,3,2,3,5,655,0,0,490,1145,GasA,...,Y,4,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,2,Attchd,2000.0,1,3,836,2,2,2,192,84,0,0,0,0,0,0,None,0,12,2008,WD,Normal,250000


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   int64  
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   int64  
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   int64  
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [89]:
nominal_data = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LandContour',
        'LotConfig', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'Foundation',
        'Heating', 'CentralAir',
        'GarageType',
        'MiscFeature',
        'SaleType', 'SaleCondition']

final_data = pd.get_dummies(df, columns = nominal_data)
final_data.head()

,Id,LotFrontage,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageYrBlt,...,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_N,CentralAir_Y,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,MiscFeature_Gar2,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,65.0,8450,3,1,2,7,5,2003,2003,196.0,2,2,3,2,1,5,706,0,0,150,856,4,4,856,854,0,1710,1,0,2,1,3,1,2,8,6,0,0,2003.0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2,80.0,9600,3,1,2,6,8,1976,1976,0.0,1,2,3,2,4,4,978,0,0,284,1262,4,4,1262,0,0,1262,0,1,2,0,3,1,1,6,6,1,2,1976.0,...,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3,68.0,11250,2,1,2,7,5,2001,2002,162.0,2,2,3,2,2,5,486,0,0,434,920,4,4,920,866,0,1786,1,0,2,1,3,1,2,6,6,1,2,2001.0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4,60.0,9550,2,1,2,7,5,1915,1970,0.0,1,2,2,3,1,4,216,0,0,540,756,3,4,961,756,0,1717,1,0,1,0,3,1,2,7,6,1,3,1998.0,...,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,5,84.0,14260,2,1,2,8,5,2000,2000,350.0,2,2,3,2,3,5,655,0,0,490,1145,4,4,1145,1053,0,2198,1,0,2,1,4,1,2,9,6,1,2,2000.0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [90]:
final_data.shape

(1460, 237)

In [91]:
dict_final_data_datatypes = final_data.columns.to_series().groupby(final_data.dtypes).groups
dict_final_data_datatypes = {k.name: v for k, v in dict_final_data_datatypes.items()}
dict_final_data_datatypes

{'float64': Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object'),
 'int64': Index(['Id', 'LotArea', 'LotShape', 'Utilities', 'LandSlope', 'OverallQual',
        'OverallCond', 'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
        'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
        'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
        'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
        'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
        'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea',
        'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
        'Fence', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
       dtype='object'),
 'uint8': Index(['MSSubClass_20', 'MSSubClass_30', '

In [92]:
# As shown above all the variables are of numeric data types
# i.e., 'uint8' , 'int64', 'float64'

#Feature Selection

In [93]:
# In this section we will plot a correlation heatmap
# But first we have to remove the transformed nominal columns from the dataset we are plotting the graph with

col_names1 = ['LotFrontage', 'MasVnrArea', 'HeatingQC', 'GarageYrBlt',
             'LotArea', 'LotShape', 'Utilities', 'LandSlope',
             'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'ExterQual',
             'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinSF1',
             'BsmtFinType2', 'BsmtFinSF2', 'BsmtExposure']
             
col_names2 = ['BsmtUnfSF', 'TotalBsmtSF', 'Electrical',
             '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
             'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
             'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
             'FireplaceQu', 'GarageFinish', 'GarageCars']
              
col_names3 = ['GarageArea', 'GarageQual',
             'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
             'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
             'Fence', 'MiscVal', 'MoSold', 'YrSold']

All_columns = ['LotFrontage', 'MasVnrArea', 'HeatingQC', 'GarageYrBlt',
             'LotArea', 'LotShape', 'Utilities', 'LandSlope',
             'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'ExterQual',
             'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinSF1',
             'BsmtFinType2', 'BsmtFinSF2', 'BsmtExposure', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical',
             '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
             'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
             'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
             'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
             'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
             'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
             'Fence', 'MiscVal', 'MoSold', 'YrSold']

corr_data = final_data[All_columns]
corr = corr_data.corr()
corr.head()

,LotFrontage,MasVnrArea,HeatingQC,GarageYrBlt,LotArea,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtExposure,BsmtUnfSF,TotalBsmtSF,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscVal,MoSold,YrSold
LotFrontage,1.000000,0.105010,0.100091,0.043020,0.100739,0.113294,0.043535,0.041567,0.176561,-0.053457,0.036853,0.078686,0.127864,-0.046807,0.100997,0.015215,-0.021636,0.076670,-0.018087,-0.009312,0.070921,0.160829,0.238274,0.003925,0.245181,0.042549,0.049981,0.220347,0.010514,-0.027856,0.120548,-0.012952,0.144494,0.034425,0.127478,0.221396,0.004560,0.044018,0.096500,0.096768,0.165229,0.201473,0.010918,0.010788,0.016838,-0.016780,0.069605,0.027366,0.023499,0.022969,0.114106,0.151755,0.008687,-0.059606,0.018942,-0.012094
MasVnrArea,0.105010,1.000000,0.153746,0.244444,0.103321,-0.089800,-0.063452,0.021083,0.407252,-0.125694,0.311600,0.176529,0.345329,-0.018065,0.297896,0.072941,0.183494,0.261256,-0.056126,-0.071330,0.175111,0.113862,0.360067,0.096851,0.339850,0.173800,-0.068628,0.388052,0.083010,0.027403,0.272999,0.199108,0.102775,-0.038450,0.282620,0.279568,0.082664,0.247015,0.274320,0.269646,0.361945,0.370884,0.136715,0.135346,0.144103,0.159991,0.122528,-0.109907,0.019144,0.062248,0.011928,0.013228,-0.096786,-0.029512,-0.006723,-0.008317
HeatingQC,0.100091,0.153746,1.000000,0.446647,0.005277,-0.114866,0.002813,0.062216,0.444388,-0.000210,0.442622,0.539398,0.502244,0.062163,0.406390,0.182710,0.154564,0.091124,-0.041304,-0.064213,0.155753,0.190882,0.263382,0.199426,0.187615,0.133855,-0.033247,0.246145,0.086195,-0.042373,0.321603,0.114962,-0.020162,-0.168474,0.489850,0.157696,0.056882,0.124507,0.205118,0.380666,0.317650,0.292921,0.161057,0.175628,0.169714,0.131475,0.166432,-0.094715,0.058389,-0.026648,-0.052111,-0.016491,-0.167478,-0.028213,0.022110,-0.010257
GarageYrBlt,0.043020,0.244444,0.446647,1.000000,-0.025865,-0.195610,0.022470,0.072131,0.514231,-0.306276,0.777182,0.616444,0.564123,-0.100144,0.590668,0.169918,0.243069,0.148782,-0.079922,-0.087684,0.237351,0.185610,0.309386,0.298086,0.224897,0.068441,-0.027095,0.219801,0.119287,-0.075424,0.466809,0.189513,-0.060155,-0.105394,0.500014,0.139519,0.112976,0.042967,0.135161,0.507948,0.474313,0.468804,0.132638,0.107502,0.242930,0.219093,0.217921,-0.284972,0.023130,-0.076181,-0.014735,-0.018503,-0.217722,-0.031779,0.004903,-0.000829
LotArea,0.100739,0.103321,0.005277,-0.025865,1.000000,-0.315484,-0.010123,-0.436868,0.105806,-0.005636,0.014228,0.013788,0.055570,0.014732,0.076829,0.033419,0.058555,0.214103,0.085312,0.111170,0.228830,-0.002618,0.260833,0.047430,0.299475,0.050986,0.004779,0.263116,0.158155,0.048046,0.126031,0.014259,0.119690,-0.017784,0.067864,0.190015,-0.025128,0.271364,0.181984,0.116326,0.154871,0.180403,0.078754,0.074381,0.015134,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.091381,-0.041423,0.038068,0.001205,-0.014261


In [94]:
# Plotting Correlation Heatmap
figure = ff.create_annotated_heatmap(
    z=corr.loc[col_names3].values,
    x=All_columns,
    y=col_names3,
    annotation_text=corr.loc[col_names3].round(1).values,
    showscale=True)
figure.show()

In [95]:
# Plotting Correlation Heatmap
figure = ff.create_annotated_heatmap(
    z=corr.loc[col_names2].values,
    x=All_columns,
    y=col_names2,
    annotation_text=corr.loc[col_names2].round(1).values,
    showscale=True)
figure.show()

In [96]:
# Plotting Correlation Heatmap
figure = ff.create_annotated_heatmap(
    z=corr.loc[col_names1].values,
    x=All_columns,
    y=col_names1,
    annotation_text=corr.loc[col_names1].round(1).values,
    showscale=True)
figure.show()

In [97]:
corr_data.columns

Index(['LotFrontage', 'MasVnrArea', 'HeatingQC', 'GarageYrBlt', 'LotArea',
       'LotShape', 'Utilities', 'LandSlope', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2',
       'BsmtExposure', 'BsmtUnfSF', 'TotalBsmtSF', 'Electrical', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold',
       'YrSold'],
      dtype='object')

#Selected Features

In [98]:
#Checking to see if there are any nan values
final_data.isnull().values.any()

False

In [99]:
final_data.isnull().sum().sum()

0

In [100]:
# is_NaN = final_data.isnull()
# row_has_NaN = is_NaN.any(axis=1)
# rows_with_NaN = final_data[row_has_NaN]

In [101]:
# rows_with_NaN

In [102]:
drop_lst = ['SalePrice', 'Id', 'GarageYrBlt',
            'KitchenQual', 'BsmtQual', 'ExterQual',
            'YearRemodAdd', 'TotalBsmtSF', 'GrLivArea',
            'BedroomAbvGr', 'KitchenQual', 'Fireplaces',
            'GarageFinish' ,'GarageCars', 'GarageCond']
X = final_data.drop(drop_lst, axis=1)
y = final_data['SalePrice']

print(X.shape)
print(y.shape)
print(type(X))
print(type(y))

(1460, 223)
(1460,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


#Data Normalization

In [103]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

X_scaled

array([[ 0.2128772 , -0.20714171,  0.70129102, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.64574726, -0.09188637,  0.70129102, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.29945121,  0.07347998, -1.01663664, ..., -0.11785113,
         0.4676514 , -0.30599503],
       ...,
       [ 0.2417352 , -0.14781027,  0.70129102, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.29945121, -0.08016039,  0.70129102, ..., -0.11785113,
         0.4676514 , -0.30599503],
       [ 0.50145724, -0.05811155,  0.70129102, ..., -0.11785113,
         0.4676514 , -0.30599503]])

#Model Development, Evaluation and Comparison

In [104]:
# Simple Linear Regression for Further feature selection
model1 = LinearRegression()

# Fit linear regression on X and y
model1.fit(X_scaled,y)

cf = model1.coef_
estimate_cf = pd.DataFrame(cf, X.columns, columns=['coef'])
estimate_cf

,coef
LotFrontage,7.218392e+02
LotArea,4.974935e+03
LotShape,-1.592088e+02
Utilities,1.490322e+03
LandSlope,1.085365e+02
...,...
SaleCondition_AdjLand,-3.177966e+15
SaleCondition_Alloca,-5.489255e+15
SaleCondition_Family,-7.066995e+15
SaleCondition_Normal,-2.333015e+16


In [129]:
# Linear Regression
# Without tuning Stochastic Gradient Descent parameters
sgdr = SGDRegressor(random_state = 1, penalty = None)
grid_param = {'eta0': [.01], 'max_iter':[1000]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
    mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0       0.009619      0.001112  ...     1336.977959                1

[1 rows x 15 columns]
Best parameters:  {'eta0': 0.01, 'max_iter': 1000}
Best result:  -1573.6323085803674
Intercept:  [281791.19321157]
                 Features  Coefficients
151    Exterior1st_CBlock  4.446829e+06
166    Exterior2nd_CBlock  4.446829e+06
103     Condition1_Artery  5.219752e+05
35          EnclosedPorch  2.985442e+05
95   Neighborhood_OldTown  2.815405e+05
..                    ...           ...
40                  Fence -1.512747e+05
77       LotConfig_Inside -1.542539e+05
66             Alley_Grvl -1.566968e+05
63            MSZoning_RM -1.647039e+05
105       Condition1_Norm -2.036823e+05

[223 rows x 2 columns]


In [ ]:
# Linear Regression
# Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter' using Grid Search
# Penalty is set to none so here we are not using any form of regulirization
sgdr = SGDRegressor(random_state = 1, penalty = None)
grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[100000, 200000, 300000, 400000]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
     mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0       21.605904      6.661982  ...    9.839828e-02                5
1       21.645519      6.694709  ...    9.839828e-02                5
2       21.546040      6.615748  ...    9.839828e-02                5
3       21.589549      6.650453  ...    9.839828e-02                5
4        0.279115      0.034791  ...    9.934991e-02                1
5        0.278688      0.034969  ...    9.934991e-02                1
6        0.279134      0.035394  ...    9.934991e-02                1
7        0.277646      0.035044  ...    9.934991e-02                1
8        0.008432      0.000681  ...    1.336978e+03                9
9        0.008443      0.000414  ...    1.336978e+03                9
10       0.008996      0.000786  ...    1.336978e+03                9
11       0.008235      0.000355  ...    1.336978e+03                9
12       0.035071      0.007606  ...    2.823437e+13           

In [ ]:
# Implementing l1 Regularization
# Tuning the SGDRegressor regularization parameters alpha using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'l1', eta0=.001, max_iter=100000)
grid_param = {'alpha': [.0001, .001, .01, .1, 1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
    mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0       0.691678      0.105007  ...        0.099350                5
1       0.685637      0.085069  ...        0.099349                4
2       0.685768      0.092210  ...        0.099345                3
3       0.688071      0.092947  ...        0.099299                2
4       0.677206      0.107871  ...        0.098966                1

[5 rows x 14 columns]
Best parameters:  {'alpha': 1}
Best result:  0.7310291028763289
Intercept:  [180920.05484765]
                 Features  Coefficients
19               1stFlrSF  18787.709580
20               2ndFlrSF  18439.837513
5             OverallQual  14961.975776
13             BsmtFinSF1  14369.289867
94   Neighborhood_NridgHt   8427.718157
..                    ...           ...
92    Neighborhood_NWAmes  -2988.997856
26           KitchenAbvGr  -3043.065052
90     Neighborhood_NAmes  -3615.098781
116       Condition2_PosN  -7453.667618
13

In [ ]:
# Implementing Elastic Net Regularization
# Tuning the SGDRegressor regularization parameters alpha and the l1 ratio using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'elasticnet', eta0=.001, max_iter=100000)
grid_param = {'alpha': [.0001, .001, .01, .1, 1], 'l1_ratio': [0,0.25,0.5,0.75,1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
     mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0        0.691574      0.103956  ...        0.099149               18
1        0.697176      0.094845  ...        0.099199               19
2        0.699445      0.094301  ...        0.099249               20
3        0.695540      0.094194  ...        0.099300               21
4        0.696762      0.089405  ...        0.099350               25
5        0.690709      0.093333  ...        0.097379               13
6        0.698848      0.089723  ...        0.097864               14
7        0.693904      0.092882  ...        0.098354               15
8        0.697825      0.087170  ...        0.098849               17
9        0.692779      0.095361  ...        0.099349               24
10       0.613648      0.092325  ...        0.082584                9
11       0.621927      0.094244  ...        0.086135               10
12       0.637612      0.111528  ...        0.090109           

#Further feature selection using Chi square

In [ ]:
scores, pvalues = chi2(X,y)
# pvalues greater than 0.01 or 1% should be excluded in the link function

chisqu = pd.DataFrame({'p_values':[a for a in pvalues]}, index=X.columns)
chisqu

,p_values
LotFrontage,0.000000e+00
LotArea,0.000000e+00
LotShape,1.000000e+00
Utilities,1.000000e+00
LandSlope,1.000000e+00
...,...
SaleCondition_AdjLand,1.000000e+00
SaleCondition_Alloca,2.953125e-51
SaleCondition_Family,1.000000e+00
SaleCondition_Normal,1.000000e+00


In [ ]:
new_df = chisqu.drop(chisqu[chisqu.p_values>0.01].index)
new_df.shape

(60, 1)

In [ ]:
lstOfFeatures = [i for i in new_df.index]
lstOfFeatures

['LotFrontage',
 'LotArea',
 'MasVnrArea',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'FireplaceQu',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'PoolQC',
 'Fence',
 'MiscVal',
 'MSSubClass_30',
 'MSZoning_C (all)',
 'MSZoning_FV',
 'Street_Grvl',
 'LandContour_HLS',
 'Neighborhood_Blmngtn',
 'Neighborhood_Crawfor',
 'Neighborhood_NoRidge',
 'Neighborhood_NridgHt',
 'Neighborhood_Somerst',
 'Neighborhood_StoneBr',
 'Neighborhood_Timber',
 'Neighborhood_Veenker',
 'Condition2_PosN',
 'Condition2_RRAn',
 'Condition2_RRNn',
 'HouseStyle_2.5Fin',
 'RoofStyle_Gambrel',
 'RoofMatl_WdShngl',
 'Exterior1st_CemntBd',
 'Exterior1st_ImStucc',
 'Exterior1st_Stucco',
 'Exterior2nd_CmentBd',
 'Exterior2nd_Other',
 'MasVnrType_Stone',
 'Foundation_Slab',
 'Foundation_Stone',
 'Heating_Floor',
 'Heating_Grav',
 'Heating_Wall',
 'CentralAir_N',
 'GarageType_Buil

In [ ]:
X = final_data[lstOfFeatures]
y = final_data['SalePrice']

print(X.shape)
print(y.shape)
print(type(X))
print(type(y))

(1460, 60)
(1460,)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [ ]:
# Normalizing numerical features so that each feature has mean 0 and variance 1
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)

X_scaled

array([[ 0.2128772 , -0.20714171,  0.51410389, ..., -0.04537649,
        -0.09103457, -0.30599503],
       [ 0.64574726, -0.09188637, -0.57075013, ..., -0.04537649,
        -0.09103457, -0.30599503],
       [ 0.29945121,  0.07347998,  0.32591493, ..., -0.04537649,
        -0.09103457, -0.30599503],
       ...,
       [ 0.2417352 , -0.14781027, -0.57075013, ..., -0.04537649,
        -0.09103457, -0.30599503],
       [ 0.29945121, -0.08016039, -0.57075013, ..., -0.04537649,
        -0.09103457, -0.30599503],
       [ 0.50145724, -0.05811155, -0.57075013, ..., -0.04537649,
        -0.09103457, -0.30599503]])

In [ ]:
# Linear Regression
# Tuning the SGDRegressor parameters 'eta0' (learning rate) and 'max_iter' using Grid Search
# Penalty is set to none so here we are not using any form of regulirization
sgdr = SGDRegressor(random_state = 1, penalty = None)
grid_param = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[100000, 200000, 300000, 400000]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
     mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0        7.198428      6.656934  ...    1.042491e-01                9
1        7.186062      6.619685  ...    1.042491e-01                9
2        7.173404      6.637197  ...    1.042491e-01                9
3        7.171307      6.624811  ...    1.042491e-01                9
4        0.091622      0.021396  ...    1.003801e-01                1
5        0.090088      0.021930  ...    1.003801e-01                1
6        0.090717      0.021791  ...    1.003801e-01                1
7        0.090334      0.021062  ...    1.003801e-01                1
8        0.011143      0.000847  ...    9.816072e-02                5
9        0.011868      0.001698  ...    9.816072e-02                5
10       0.011230      0.000773  ...    9.816072e-02                5
11       0.011117      0.000826  ...    9.816072e-02                5
12       0.004809      0.000105  ...    3.033471e+13           

In [ ]:
# Implementing l1 Regularization
# Tuning the SGDRegressor regularization parameters alpha using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'l1', eta0=.001, max_iter=100000)
grid_param = {'alpha': [.0001, .001, .01, .1, 1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
    mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0       0.138495      0.034079  ...        0.100380                5
1       0.143951      0.025761  ...        0.100380                4
2       0.143230      0.025549  ...        0.100379                3
3       0.142993      0.025023  ...        0.100369                2
4       0.143212      0.025275  ...        0.100270                1

[5 rows x 14 columns]
Best parameters:  {'alpha': 1}
Best result:  0.7441598807682835
Intercept:  [180913.81872158]
                 Features  Coefficients
9                2ndFlrSF  22342.235871
8                1stFlrSF  20001.943152
30   Neighborhood_NridgHt  14246.671150
29   Neighborhood_NoRidge  11260.172352
12             GarageArea  11235.105264
4              BsmtFinSF1   9235.602882
3            BsmtFinType1   9109.870490
32   Neighborhood_StoneBr   9080.952441
7               BsmtUnfSF   8817.899088
11            FireplaceQu   6457.305927
40

In [ ]:
# Implementing Regularization
# Tuning the SGDRegressor regularization parameters alpha and the l1 ratio using Grid Search
sgdr = SGDRegressor(random_state = 1, penalty = 'elasticnet', eta0=.001, max_iter=100000)
grid_param = {'alpha': [.0001, .001, .01, .1, 1], 'l1_ratio': [0,0.25,0.5,0.75,1]}

gd_sr = GridSearchCV(estimator=sgdr, param_grid=grid_param, scoring='r2', cv=5)

gd_sr.fit(X_scaled, y)

results = pd.DataFrame.from_dict(gd_sr.cv_results_)
print("Cross-validation results:\n", results)

best_parameters = gd_sr.best_params_
print("Best parameters: ", best_parameters)

best_result = gd_sr.best_score_ # Mean cross-validated score of the best_estimator
print("Best result: ", best_result)

best_model = gd_sr.best_estimator_
print("Intercept: ", best_model.intercept_)

print(pd.DataFrame(zip(X.columns, best_model.coef_), 
                   columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Cross-validation results:
     mean_fit_time  std_fit_time  ...  std_test_score  rank_test_score
0        0.137569      0.033802  ...        0.100324               16
1        0.147868      0.028218  ...        0.100338               17
2        0.146066      0.024310  ...        0.100352               18
3        0.146682      0.028650  ...        0.100366               19
4        0.143757      0.025766  ...        0.100380               23
5        0.143782      0.024990  ...        0.099826               11
6        0.144430      0.024747  ...        0.099963               12
7        0.144571      0.025356  ...        0.100101               13
8        0.145213      0.025990  ...        0.100240               14
9        0.143534      0.025363  ...        0.100380               22
10       0.134328      0.038474  ...        0.095414                7
11       0.140104      0.030657  ...        0.096549                8
12       0.143634      0.026242  ...        0.097727           